In [339]:
from bs4 import BeautifulSoup
import requests
from cleantext import clean

In [340]:
url = "https://eng.yidaiyilu.gov.cn/list/w/know_about"
req = requests.get(url)
soup = BeautifulSoup(req.text, "html.parser")
# setup
all_links = soup.ul
# all the links in the <ul> tag on the yidaiyilu site. This contains all the sources they link in their online list. 
all_links = str(all_links)
# turning into a string object instead of a bs4 object. 
all_content_list = all_links.split(" ")
# splitting based on spaces, this is just to make it into a legible list. 
links_list = []
i = 0
for value in all_content_list: 
    # looping through the list, building a new list with only the links. 
    if "href" in value: 
        links_list.append(value)
links_cleaned_list = []
for value in links_list: 
    # cleaning the links-- if it doesn't have a full https in front of it already adding it, removing the html href tag. 
    if "https://eng.yidaiyilu.gov.cn" not in value:
        full_link = "https://eng.yidaiyilu.gov.cn"+value[5:]
        links_cleaned_list.append(full_link.replace('"', ''))
    if value[-1] != "l":
        value+"l"
    else:
        links_cleaned_list.append(value[5:])
print(links_cleaned_list)

['https://eng.yidaiyilu.gov.cn/p/0H647R18.html', 'https://eng.yidaiyilu.gov.cn/p/0PPN0HMN.html', 'https://eng.yidaiyilu.gov.cn/p/0N4P7CF0.html', 'https://eng.yidaiyilu.gov.cn/p/006Q9GBH.html', 'https://eng.yidaiyilu.gov.cn/p/0CESAH1N.html', 'https://eng.yidaiyilu.gov.cn/p/041D0RIJ.html', 'https://eng.yidaiyilu.gov.cn/p/86739.html', 'https://eng.yidaiyilu.gov.cn/p/46076.html', 'https://eng.yidaiyilu.gov.cn/p/34863.html', 'https://eng.yidaiyilu.gov.cn/p/30277.html', 'https://eng.yidaiyilu.gov.cn/p/16639.html', 'https://eng.yidaiyilu.gov.cn/p/13754.html', 'https://eng.yidaiyilu.gov.cn/p/12731.html', 'https://eng.yidaiyilu.gov.cn/p/12479.html', 'https://eng.yidaiyilu.gov.cn/p/10477.html', 'https://eng.yidaiyilu.gov.cn/p/1084.html']


In [341]:
# for each link:
# 
# 1. create a new file (open())
# 2. write the text content into the file
# 3. close the file.

In [346]:
for link in links_cleaned_list:
  print(link)

https://eng.yidaiyilu.gov.cn/p/0H647R18.html
https://eng.yidaiyilu.gov.cn/p/0PPN0HMN.html
https://eng.yidaiyilu.gov.cn/p/0N4P7CF0.html
https://eng.yidaiyilu.gov.cn/p/006Q9GBH.html
https://eng.yidaiyilu.gov.cn/p/0CESAH1N.html
https://eng.yidaiyilu.gov.cn/p/041D0RIJ.html
https://eng.yidaiyilu.gov.cn/p/86739.html
https://eng.yidaiyilu.gov.cn/p/46076.html
https://eng.yidaiyilu.gov.cn/p/34863.html
https://eng.yidaiyilu.gov.cn/p/30277.html
https://eng.yidaiyilu.gov.cn/p/16639.html
https://eng.yidaiyilu.gov.cn/p/13754.html
https://eng.yidaiyilu.gov.cn/p/12731.html
https://eng.yidaiyilu.gov.cn/p/12479.html
https://eng.yidaiyilu.gov.cn/p/10477.html
https://eng.yidaiyilu.gov.cn/p/1084.html


In [342]:
# Getting the raw text data
f = open("demo.txt", "w")
""""
setup
"""
link = "https://eng.yidaiyilu.gov.cn/p/0H647R18.html"
link_req = requests.get(link).text.encode("utf8").decode("ascii", "ignore")
page_soup = BeautifulSoup(link_req, "html.parser")

article_content = page_soup.find_all("p")
# everything with a <p> tag
all_text = page_soup.find_all(string = True)
# all the string content; useful for getting the date updated, source, editor
title = page_soup.find_all("h1")

# f.write(str(link))
# f.write(str(article_content))
# f = open("title.txt", "w")
# f.write(str(text))
# all_text
# title
# title=str(title).split("<" or ">")
# title=list(title)

In [343]:
"""
Fixing up the Updated, Source fields
"""
for x in range(len(all_text)):
  if "Source:" in all_text[x]: f.write(str(all_text[x]+ " "+ all_text[x+1])+" "+"\n")
    # print(all_text[x] + " "+ all_text[x+1])
    # print(all_text[x] + " "+ all_text[x+1])
  elif "Updated:" in all_text[x]: f.write(str(all_text[x]+ " "+ all_text[x+1]+" "+"\n"))
    # print(all_text[x]+ " "+all_text[x+1])
  elif "Editor:" in all_text[x]: f.write(+str(all_text[x]+" "+ all_text[x+1]+" "+"\n"))
    # print(all_text[x]+" "+all_text[x+1])
    # f.write(str(all_text[x]+" "+all_text[x+1]))
    # print(all_text[x]+ " "+all_text[x+1])


In [344]:
"""
cleaning up the <p> content
1. turn article_content into a list
2. turn list contents into strings
3. if not in there, write it into file, else don't.

"""
article_content_list = list(article_content)

for content in article_content_list: 
  content = str(content)
  if("data" not in content):
    content = content.replace("<p>", "").replace("</p>", "")
    f.write(content + "\n")
